<a href="https://colab.research.google.com/github/ErikSeras/usos_r_python/blob/main/analisis_espacial/005_generar_gif_no2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ¿Cómo observar la variación de $NO_2$ en Lima Metropolitana?

Elaborado por: __Erik Seras__ (_er.seras.perez@gmail.com_)

Fecha: __2020-11-09__

Como ejemplo, se quiere ver el desarrollo de las concentraciones de $NO_2$ en Lima Metropolitana. Una de las formas de alcanzar ese objetivo es mediante la creación de un GIF que permita visualizar tales concentraciones.

Los insumos necesarios serán:
- `earthengine-api`: vinculaciones de Python y JavaScript para llamar a la API de Earth Engine.
- `geemap`: un paquete de Python para mapeo interactivo con Google Earth Engine, ipyleaflet e ipywidgets)
- `Sentinel-5P NRTI NO2: Near Real-Time Nitrogen Dioxide`: este dataset proporciona imágenes de alta resolución casi en tiempo real de las concentraciones de $NO_2$.

# 1 Acciones previas

## 1.1 Instalar el paquete `geemap`

In [ ]:
## Instalar geemap
! pip install geemap

## 1.2 Importar paquetes

In [51]:
import ee
import geemap.eefolium as geemap
import datetime
import pandas as pd

## 1.3 Iniciar sesión de Google Earth Engine

In [ ]:
# Iniciar sesión de GEE
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# 2 Crear el GIF con imágenes promedio mensuales de valores de $NO_2$ en Lima Metropolitana

## 2.1 Función que permite obtener imágenes mensuales

In [53]:
"""
Elaborado por Erik Seras (2020-09-11)

La función monthly_images permite generar una imagen por cada mes
de un ImageCollection.

Los argumentos son:
coleccion: ee.ImageCollection que se quiere procesar
roi: región de interés, se usará para cortas la imagen final mediante clip()
banda: seleccionar las bandas a analizar ['B1'] o ['B1', 'B2', ...]
start_date: fecha de inicio del análisis
end_date: fecha de fin del análisis
"""

def monthly_images(coleccion, roi, banda, start_date = '2018-09-01', end_date = '2020-09-01'):
    """
    La función generar_no2_mensual genera una imagen promediando los valores
    de los pixeles disponibles en un roi dado
    """
    def generar_imagen_mensual(date):
        """
        La función cortar_imagen corta los las imágenes disponibles
        basada en un roi dado
        """
        def cortar_imagen(image):
            return image.clip(roi)
        # Fecha de inicio del mes a analizar
        start = ee.Date(date)
        # Fecha de fin del mes a analizar
        end = ee.Date(date).advance(1, 'month')
        # Creación de un rango de datos
        date_range = ee.DateRange(start, end)
        # Generar una imagen mensual de no2 usando la media
        image_no2_mensual = coleccion.select(banda) \
        .filterDate(date_range).filterBounds(roi) \
        .map(cortar_imagen).mean()

        return image_no2_mensual

    """
    La función make_datelist crea una lista de fechas en la que se aumenta
    n meses a la fecha de inicio
    """
    def make_datelist(n):
        return Date_Start.advance(n,'month')

    # Fecha de inicio del análisis
    Date_Start = ee.Date(start_date)
    # Fecha de fin del análisis
    Date_End = ee.Date(end_date)
    # Diferencia de meses entre la fecha de inicio y fin
    n_months = Date_End.difference(Date_Start,'month').round()
    # Crear una lista que número enteros
    dates = ee.List.sequence(0,n_months,1)
    # Usar la función make_datelist para generar imagenes mensuales
    dates = dates.map(make_datelist)

    # Usar la función generar_imagen_mensual
    list_images = dates.map(generar_imagen_mensual)

    # Retornoa un ImageCollection con las imagenes mensuales creadas
    return ee.ImageCollection(list_images)

## 2.2 Generación y descarga del GIF

In [ ]:
# Contorno del GIF
aoi = ee.Geometry.Polygon(
    [[[-77.430, -11.400],
    [-77.430, -12.700],
    [-76.450, -12.700],
    [-76.450, -11.400]]], None, False
)

# Lima Metropolitana
lima_metropolitana = ee.FeatureCollection("FAO/GAUL/2015/level2") \
    .filterMetadata("ADM2_NAME", "equals", "Lima")

# Dataset Sentinel 5P TROPOMI NO2
S5P_no2 = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_NO2')

# Imagenes mensuales promedio de NO2 en Lima Metropolitana
lima_metropolitana_no2 = monthly_images(
    coleccion = S5P_no2,
    roi = lima_metropolitana,
    banda = ['NO2_column_number_density'],
    start_date = '2018-09-01', end_date = '2020-09-01'
)

# Argumentos para el GIF
videoArgs = {
    'dimensions': 1024,
    'region': aoi,
    'framesPerSecond': 5,
    'crs': 'EPSG:3857',
    'min': 0,
    'max': 0.0003,
    # Paleta de inferno de matplotlib
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

# Dirección del GIF a generar
saved_gif = "no2_mensual.gif"

# Descargar el GIF
geemap.download_ee_video(lima_metropolitana_no2, videoArgs, saved_gif)

# 3 Diseño del GIF final

## 3.1 Contenido a agregar al GIF

In [47]:
# Argumentos para geemap.add_text_to_gif()

# Fecha de mes para añadir al GIF
list_mes_no2 = (
    pd.date_range('2018-09-01','2020-10-01' , freq='1M')-pd.offsets.MonthBegin(1)
).strftime("%Y-%B").tolist()

# Título general del GIF
titulo = []

for i in range(0, len(list_mes_no2)):
    titulo_fecha = "Dióxido de nitrógeno en Lima Metropolitana " +list_mes_no2[i]
    titulo.append(titulo_fecha)

# Nombre del autor del GIF
autor = "Elaborado por: Erik Seras" # Poner su nombre

# Color del texto
color_texto = '#9ef0ff' # Formato hex

# Tamaño de la fuente
tamano_texto = 24

# Tener en cuenta el argumente framesPerSecond para crear el GIF en la
# sección 2.2, se debe hacer un arreglo para que la el flujo de fotogramas
# base del GIF no se altere al usar la función geemap.add_text_to_gif()
fps_original = 5 # fps
mantener_fps = 100/fps_original*10

## 3.2 Maquetación final del GIF

Generar los nombres de los GIF a usar.

In [48]:
# GIF descargado
in_gif = 'no2_mensual.gif'
# GIF modificado
out_gif = 'no2_mensual_final.gif'

Añadir el texto necesario para dar un mejor entendimiento al GIF.

In [49]:
# Función para añadir texto a un GIF
geemap.add_text_to_gif(
    # Dirección del gif inicial y saliente
    in_gif, out_gif,
    # Texto que se desea agregar al GIF
    text_sequence = titulo,
    # Ubicación del texto en el GIF
    xy = ('10%', '5%'),
    # Color y tamaño del texto
    font_color = color_texto, font_size = tamano_texto,
    # Velocidad de cada fotograma en milisegundos
    duration = mantener_fps
)

# Función para añadir texto a un GIF
geemap.add_text_to_gif(
    # Dirección del gif inicial y saliente
    out_gif, out_gif,
    # Ubicación del texto en el GIF
    xy=('55%', '92%'),
    text_sequence = autor,
    # Color y tamaño del texto
    font_color = color_texto, font_size = tamano_texto,
    # Velocidad de cada fotograma en milisegundos
    duration = mantener_fps
)

In [ ]:
# Mostrar el GIF final
geemap.show_image(out_gif)

# Bibliografía

> Copernicus Sentinel-5P (processed by ESA), 2018, TROPOMI Level 2 Nitrogen Dioxide total column products. Version 01. European Space Agency. https://doi.org/10.5270/S5P-s4ljg54

> Gorelick, N., Hancher, M., Dixon, M., Ilyushchenko, S., Thau, D., & Moore, R. (2017). Google Earth Engine: Planetary-scale geospatial analysis for everyone. Remote sensing of Environment, 202, 18-27. https://doi.org/10.1016/j.rse.2017.06.031

> Wu, Q., Lane, C. R., Li, X., Zhao, K., Zhou, Y., Clinton, N., DeVries, B., Golden, H. E., & Lang, M. W. (2019). Integrating LiDAR data and multi-temporal aerial imagery to map wetland inundation dynamics using Google Earth Engine. Remote Sensing of Environment, 228, 1-13. https://doi.org/10.1016/j.rse.2019.04.015 ([pdf](https://gishub.org/2019_rse) | [source code](https://doi.org/10.6084/m9.figshare.8864921))

> Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305